In [72]:
## import data processing/cleaning , data modeling libraries
#!pip install boto3

import pandas as pd
import os
import sys
import re as re
import datetime as datetime
import numpy as np
import collections
import boto3
import io
import string

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from nltk.tag.perceptron import PerceptronTagger

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import silhouette_score
    
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from gensim.models import doc2vec
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models import fasttext

import warnings
warnings.filterwarnings('ignore')

source="/home/ubuntu/Notebook/adviewRecomm/version3/temp/"
sparkfiles = "/home/ubuntu/Notebook/spark-modules/clust_vtitleTrans/"
destination=source

lang=["en"] # video language got from video title using python Library
clustSize = 0 # no of clusters - initialization
t0 = datetime.datetime.now()
print(source)

/home/ubuntu/Notebook/adviewRecomm/version3/temp/


In [73]:
## outData41 - video data coming from postgres DB
#  outData41.to_csv(source+"youtubeModelData_stats1.csv",index=False)

outData43 = pd.read_csv(source+"clipVdChanelData_model_in.csv",encoding="utf-8")
print(outData43.shape)
print(outData43.head(n=2)) # 39:
outData43 = outData43.drop_duplicates()
outData43 = outData43.dropna(subset=["v_title_trans"])
print(outData43.shape)

(215482, 32)
                v_id                          v_title  \
0  10155398318101089  How it's made Electrical Cables   
1  10155398314226089       How It's Made Racing Karts   

                     v_title_trans ch_lang_raw  Unnamed: 0  ch_id v_descr  \
0  How it's made Electrical Cables     english         NaN    NaN     NaN   
1       How It's Made Racing Karts     english         NaN    NaN     NaN   

   v_seq_id v_lang_raw v_thumbnail    ...      ch_is_clickable v_stat_viewCnt  \
0       NaN        NaN         NaN    ...                  NaN            NaN   
1       NaN        NaN         NaN    ...                  NaN            NaN   

  v_stat_likeCnt  v_stat_dislikeCnt v_stat_cmntCnt v_stat_status  \
0            NaN                NaN            NaN           NaN   
1            NaN                NaN            NaN           NaN   

  v_stat_updatStamp lang_vtitle videoPubDate videoPubYr  
0               NaN         NaN          NaN        NaN  
1               Na

In [74]:
## NLP PRE-PROCESSING

#text=" the JanaSena Party Formation Day celebrations || LIVE funny celebration|| Pawan Kalyan || Guntur"
text="Chak De India | Full Title Song | Shah Rukh Khan | Sukhvinder Singh | Salim | Marianne D'Cruz"

def nltk_clean_sent(line):
    if len(line)>0:
        ## remove the punctuation/emoticons/digits/multispaces with single from the line
        ## dont make lowercase before the pos tagging
        line_lower = line.strip()
        line_punct = re.sub('['+string.punctuation+']',' ',line_lower)
        line_emots = re.sub(r'[\u200b-\u2fff]+',' ',line_punct)
        line_digis = re.sub(r'[0-9]+',' ',line_emots)
        line_spaces = re.sub(r'[\s]+',' ',line_digis)
        line = line_spaces
    return line

def nltk_extract_postags(line):
    cleaned_str = ''
    tokens = nltk.word_tokenize(line)
    tokens_pos = PerceptronTagger().tag(tokens)
    #print(tokens_pos)
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
    
    for tag_word in tokens_pos:
        if tag_word[1] in nltk_tags:
            if len(tag_word[0])>2:
                cleaned_str += tag_word[0]+' '
    return cleaned_str.strip().lower()

def nltk_apply_lemma(line):
    tokens_lemmas = [WordNetLemmatizer().lemmatize(word) for word in line.split()]
    ## stemming
    tokens_stops = [word for word in tokens_lemmas if word not in stopwords.words('english')]
    tokens_stops = [word for word in tokens_stops if len(word.strip())>2]
    tokens_stops = list(set(tokens_stops))
    return tokens_stops

def nltk_extract_tags(line):
        ## tokenize the sentence/get tokens that contains only letters
        line_clean = nltk_clean_sent(line)
        ## apply postags to the words and get only couple of tags and word length >2
        tokens_pos = nltk_extract_postags(line_clean)
        ## apply lemmatize/stemming and remove stopwords
        token_lemma = nltk_apply_lemma(tokens_pos)
        return token_lemma
    
nltk_extract_tags(text)

['rukh',
 'marianne',
 'full',
 'india',
 'salim',
 'singh',
 'chak',
 'title',
 'song',
 'shah',
 'cruz',
 'sukhvinder',
 'khan']

In [75]:
# TF-IDF transformation
def getTfIdfMetrics(outData44,lang):
    outData44_en = outData44[outData44.ch_lang_raw==lang]
    t0 = datetime.datetime.now()
    tfidf_transform = TfidfVectorizer(tokenizer=nltk_extract_tags,min_df=6,max_df=0.9,stop_words='english',use_idf=True,ngram_range=(1,1))
    #terms = tfidf_transform.get_feature_names()
    #print(len(terms))
    #print(terms[:100])
    tfidf_vecto = tfidf_transform.fit_transform(outData44_en["v_title_trans"])
    print(tfidf_vecto.shape)
    # got shape of (83806, 149542) - without translation
    t1 = datetime.datetime.now()
    print("END TIME after TFIDF =",t1)
    print("time taken until TFIDF=",(t1-t0))
    return tfidf_vecto

## PRINTING FIRST 100 FEATURES
#getTfIdfMetrics(outData44,'telugu')
#terms = tfidf_transform.get_feature_names()
#print(terms[:100])

In [76]:
# ## DOC2VEC model
# #outData44_en_sample = outData44_en[50:55]

# cleaneddoc = [nltk_extract_tags(text) for text in list(outData44_en["v_title_trans"])]
# taggeddoc = [doc2vec.TaggedDocument(val,[idx]) for idx,val in enumerate(cleaneddoc)]

# d2vmodel = Doc2Vec(size=100,min_count=2,alpha=0.025,min_alpha=0.025)
# d2vmodel.build_vocab(taggeddoc)
# #print(d2vmodel[0])
# d2vmodel.train(taggeddoc,total_examples=d2vmodel.corpus_count,epochs=10,start_alpha=0.002,end_alpha=-0.016)
# d2vmodel_vecs = [d2vmodel.infer_vector(val) for idx,val in enumerate(cleaneddoc)]
# #print(d2vmodel_vecs[0])

##feature_vecto = d2vmodel.docvecs.doctag_syn0
##print(len(d2vmodel_vecs))

## WORD2VEC model # Word2Vec
def getWord2VecMetrics(outData44,lang):
    outData44_en = outData44[outData44.ch_lang_raw==lang]
    t0 = datetime.datetime.now()
    cleaneddoc = [nltk_extract_tags(text) for text in list(outData44_en["v_title_trans"])]
    cleaneddoc = [text for text in cleaneddoc if len(text)>0]
    w2vmodel = Word2Vec(cleaneddoc,min_count=1,size=200)
    w2v_vecs = w2vmodel.wv.syn0
    t1 = datetime.datetime.now()
    print("END TIME after TFIDF =",t1)
    print("time taken until TFIDF=",(t1-t0))
    return w2v_vecs
#feature_vecto = getWord2VecMetrics(outData44,lang)
#print(feature_vecto.shape)

In [77]:
## MODEL FUNCTION DEFINITION
#feature_vecto = np.stack(d2vmodel.docvecs)
def getModelDataTfIdf(outData5,lang,feature_vecto):
    ## chosing no of clusters
    if len(outData5) < 1800:
        clustSize=3
    else:
        clustSize = int(np.round(np.divide(len(outData5),1800)))
    print("cluster lang=",lang)    
    print("clust data len=",len(outData5))
    print("cluster size=",clustSize)

    ## KMeans model
    from sklearn.cluster import KMeans
    #kmeans = KMeans(n_clusters=clustSize,init='k-means++',max_iter=80).fit(feature_vecto)
    kmeans = KMeans(n_clusters=clustSize).fit(feature_vecto)
    preds1 = kmeans.labels_

    ### NO worries on this PART ###
    print(collections.Counter(preds1))
    b=pd.DataFrame(preds1)
    clustData1=pd.DataFrame(outData5)
    clustData1["clustNum"]=pd.Series(preds1)

    print(clustData1.head(n=1))
    print(clustData1.shape)
    #clustData1.to_csv(destination+"/clustData_"+lang+".csv",index=False)
    ##clustData.groupby(["chCategory","clustNo","v_title"]).size().sort_values(ascending=False)
    #print(clustData1.columns)
    return clustData1

In [78]:
## LANGUAGE LEVEL CLUSTERING
startT = datetime.datetime.now()
#langs = ['english','hindi','bengali','telugu','tamil']
langs = ['telugu']
for lang in langs:
        outData5 = outData43[outData43.ch_lang_raw==lang]
        ## Dataframe Length Check
        if(len(outData5)>0):
            print(outData5.shape)
            
            ## feature engineering
            ##feature_vecto = getTfIdfMetrics(outData5,lang)
            feature_vecto = getWord2VecMetrics(outData5,lang)
            ## Model Data Function Call
            clustData2 = getModelDataTfIdf(outData5,lang,feature_vecto)
            print(clustData2.shape)
endT = datetime.datetime.now()
print("TIME taken for Modeling=",(endT-startT))
## 160 records - 36 min time with tfidf kmeans

(33808, 32)
END TIME after TFIDF = 2018-07-05 10:56:47.583108
time taken until TFIDF= 0:01:15.431852
cluster lang= telugu
clust data len= 33808
cluster size= 19
Counter({0: 14996, 11: 3577, 5: 1625, 2: 799, 10: 339, 12: 265, 18: 157, 13: 114, 16: 61, 6: 47, 1: 41, 4: 26, 14: 21, 17: 20, 3: 19, 7: 17, 9: 17, 15: 12, 8: 8})
                v_id                                            v_title  \
987  144531536255932  mrigiNcin yaalukltoo vaarN loonee mimmlni miir...   

                          v_title_trans ch_lang_raw  Unnamed: 0  ch_id  \
987  mrigiNcc in the world of your life      telugu         NaN    NaN   

    v_descr  v_seq_id v_lang_raw v_thumbnail   ...     v_stat_viewCnt  \
987     NaN       NaN        NaN         NaN   ...                NaN   

    v_stat_likeCnt v_stat_dislikeCnt  v_stat_cmntCnt v_stat_status  \
987            NaN               NaN             NaN           NaN   

    v_stat_updatStamp lang_vtitle videoPubDate videoPubYr clustNum  
987               N

In [79]:
# ## WORD2VEC model # Word2Vec
# def getWord2VecMetrics(outData44,lang):
# outData44_en = outData44[outData44.ch_lang_raw==lang]
# t0 = datetime.datetime.now()
# cleaneddoc = [nltk_extract_tags(text) for text in list(outData44_en["v_title_trans"])]
# cleaneddoc = [text for text in cleaneddoc if len(text)>0]
# w2vmodel = Word2Vec(cleaneddoc,min_count=1,size=200)
# w2v_vecs = w2vmodel.wv.syn0
# t1 = datetime.datetime.now()
# print("END TIME after w2v =",t1)
# print("time taken until w2v=",(t1-t0))
# return w2v_vecs, w2vmodel
# # outData44.ch_lang_raw.unique()
# feature_vecto , w2vmodel= getWord2VecMetrics(outData44,'english')
# # print(feature_vecto.shape)

# word_vectors = feature_vecto

# import time

# from sklearn.cluster import KMeans

# n_words = word_vectors.shape[0]
# vec_size = word_vectors.shape[1]
# print("#words = {0}, vector size = {1}".format(n_words, vec_size))

# start = time.time()
# print("Compute clustering ... ", end="", flush=True)
# kmeans = KMeans(n_clusters=10, n_jobs=-1, random_state=0)
# idx = kmeans.fit_predict(word_vectors)
# print("finished in {:.2f} sec.".format(time.time() - start), flush=True)

# import matplotlib.pyplot as plt
# %matplotlib inline

# inertia = []
# for k in range(1, 20):
# kmeans = KMeans(n_clusters=10, n_jobs=-1, random_state=0)
# # idx = kmeans.fit_predict(word_vectors)

# kmeans = KMeans(n_clusters=k, random_state=1).fit(word_vectors)
# inertia.append(np.sqrt(kmeans.inertia_))

# plt.plot(range(1, 20), inertia, marker='s');
# plt.xlabel('$k$')
# plt.ylabel('$J(C_k)$');

In [80]:
#     from sklearn.cluster import k_means_
#     from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances

#     # Manually override euclidean
#     def euc_dist(X, Y = None, Y_norm_squared = None, squared = False):
#         #return pairwise_distances(X, Y, metric = 'cosine', n_jobs = 10)
#         return cosine_similarity(X, Y)
#     k_means_.euclidean_distances = euc_dist

#     kmeans = k_means_.KMeans(n_clusters = clustSize,random_state = 3425).fit(tfidf_vecto)